# Retrieval exercise

In this exercise, you will implement the query likelihood model with Jelinek-Mercer smoothing. This assignment builds on the previous assignment for creating a Pyserini index.


## 1. Build the index
Download the MS MARCO passage collection and build an index using [Pyserini](https://github.com/castorini/pyserini).
This code is similar to PART 1 of the indexing assignment.

In [17]:
!pip install pyserini
!pip install faiss-cpu

!git clone https://github.com/castorini/anserini.git --recurse-submodules

!wget https://msmarco.blob.core.windows.net/msmarcoranking/collection.tar.gz -P data/msmarco_passage/
!tar xvfz data/msmarco_passage/collection.tar.gz -C data/msmarco_passage

!cd anserini && python tools/scripts/msmarco/convert_collection_to_jsonl.py \
 --collection-path ../data/msmarco_passage/collection.tsv --output-folder ../data/msmarco_passage/collection_jsonl

!rm data/msmarco_passage/*.tsv
!rm -rf sample_data

!python -m pyserini.index.lucene -collection JsonCollection -generator DefaultLuceneDocumentGenerator -threads 9 \
-input data/msmarco_passage/collection_jsonl -index indexes/lucene-index-msmarco-passage -storePositions -storeDocvectors -storeRaw



fatal: destination path 'anserini' already exists and is not an empty directory.
--2023-09-19 10:53:22--  https://msmarco.blob.core.windows.net/msmarcoranking/collection.tar.gz
Resolving msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)... 20.150.34.4
Connecting to msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)|20.150.34.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1035009698 (987M) [application/octet-stream]
Saving to: ‘data/msmarco_passage/collection.tar.gz.1’

collection.tar.gz.1 100%[===================>] 987.06M  12.3MB/s    in 85s     

2023-09-19 10:54:48 (11.6 MB/s) - ‘data/msmarco_passage/collection.tar.gz.1’ saved [1035009698/1035009698]

collection.tsv
Converting collection...
Converted 0 docs, writing into file 1
Converted 100,000 docs, writing into file 1
Converted 200,000 docs, writing into file 1
Converted 300,000 docs, writing into file 1
Converted 400,000 docs, writing into file 1
Converted 500,000 docs, writing

## 2. Download and read the query file
You will rank MSMARCO passages for this set of queries.

In [18]:
!wget http://gem.cs.ru.nl/IR-Course/queries.txt

queries = dict()
with open("queries.txt", "r") as f:
    for line in f:
        cols = line.split("\t")
        queries[cols[0].strip()] = cols[1].strip()

--2023-09-19 11:12:01--  http://gem.cs.ru.nl/IR-Course/queries.txt
Resolving gem.cs.ru.nl (gem.cs.ru.nl)... 131.174.31.31
Connecting to gem.cs.ru.nl (gem.cs.ru.nl)|131.174.31.31|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2275 (2.2K) [text/plain]
Saving to: ‘queries.txt.1’

queries.txt.1       100%[===================>]   2.22K  --.-KB/s    in 0s      

2023-09-19 11:12:02 (291 MB/s) - ‘queries.txt.1’ saved [2275/2275]



## 3. Implement the retrieval model
You will implement language model with Jelinek-Mercer (JM) smoothing:
$$score(q,d) = \sum_{t \in q} log ((1-\lambda) \frac{c(t, d)}{|d|} + \lambda \frac{c (t, C)}{|C|}),$$
where $c (t, d)$ and $c (t, C)$ represent frequency of a term in a document and collection, respectively.

**Notes about your implementation:**
- Skip a term if it does not exist in the whole collection. This avoids $log(0)$.
- Make sure to use the right form of a query (analyzed vs. not analyzed)
- Use natural logarithm


### 3.1. Obtain collection length
In this code, the global variable `len_C` denotes collection length.

In [19]:
from pyserini.index.lucene import IndexReader

global len_C

# =======Your code=======
index_reader = IndexReader('indexes/lucene-index-msmarco-passage')
len_C = index_reader.stats()['total_terms']
# =======================


Run this to test your code. If everything is correct, you should not get errors here.

In [20]:
assert len_C == 352316036

### 3.2. Obtain document length

Here you need compute the length of document (as it is stored in the index).

*Hint: You first need to get the document vector from your Pyserini index. Consult [Pyseniri documentation](https://github.com/castorini/pyserini/blob/master/docs/usage-indexreader.md) to find the right function.*

In [21]:
def len_doc(d):
    # =======Your code=======
    doc_vec = index_reader.get_document_vector(d)
    len_d = sum(doc_vec.values())
    # =======================
    return len_d

In [22]:
# Test your code
assert len_doc("2674124") == 31

### 3.3. Obtain collection frequency of a term

Obtain number of times a term appers in the whole collection.

In [23]:
def coll_freq(t):
    # =======Your code=======
    df, cf = index_reader.get_term_counts(t)
    # =======================
    return cf

In [24]:
# Test your code
assert coll_freq("record") == 226439

### 3.4. Obtain term frequency

Obtain number of times a term appears in a document.

In [25]:
def term_freq(t, d):
    # =======Your code=======
    doc_vec = index_reader.get_document_vector(d)
    tf = doc_vec[t] if t in doc_vec else 0
    # =======================
    return tf

In [26]:
# Test your code
assert term_freq("record", "2674124") == 2
assert term_freq("presence", "2674124") == 0

### 3.5. Compute JM-smoothed probability for a single term

Here, you need to implement the following formula:

$$P_{JM}(t,d) = (1-\lambda) \frac{c(t, d)}{|d|} + \lambda \frac{c (t, C)}{|C|}$$

In [27]:
def prob_t_Md(t, d, lambd):
    # =======Your code=======
    p_t_Md = ((1-lambd)*(term_freq(t,d)/len_doc(d)))+(lambd*(coll_freq(t)/len_C))
    # =======================
    return p_t_Md


In [28]:
# Test your code
assert prob_t_Md("record", "2674124", 0.1) == 0.05812878768549357
assert prob_t_Md("darcig", "2674124", 0.1) == 0


### 3.6. Compute JM-smoothed probability for a query

In [29]:
import math

def score_doc(q, d, lambd):
    # =======Your code=======
    p_q_Md = 0
    for term in q:
      # Skip term analysis:
      df, cf = index_reader.get_term_counts(term, analyzer=None)
      if cf == 0 :
        continue
      p_q_Md += math.log(prob_t_Md(term,d,lambd))
    # =======================
    return p_q_Md


In [33]:
q1 = index_reader.analyze("are naturalization records public")
q2 = index_reader.analyze("kemeet land")
doc = "2674124"
assert score_doc(q1, doc, 0.1) == -9.227787624348021
assert score_doc(q2, doc, 0.1) == -10.254756777887694

## 4. Rank documents for the given queries
Ranking is done in two steps:
1. First pass retrieval: Use a fast ranker (i.e., Pyserini LuceneSearcher) ro rank all documents for a given query.
2. Second pass retrieval: Re-rank top-100 documents from the 1st pass retrieval using your retrieval model. This is to make the ranking process efficient.

**Notes:**
- You need to change the default values of LuceneSearcher functions to obtain top-100 documents
- Set the value of lambda to 0.1
- Store your final ranking results in the `results` variable. Every item in the `results` list is a list containing queryID, documentID, and score. This is an example how the content of results should look like:

`[['23849', '4348282', -10.65],
 ['23849', '7119957', -12.63],
 ['23849', '', -17.687729001682484],
 ...]`

In [41]:
from pyserini.search.lucene import LuceneSearcher
lambd = 0.1
results = []
searcher = LuceneSearcher("indexes/lucene-index-msmarco-passage")
for qid, q in queries.items():
    # =======Your code=======
    hits = searcher.search(q,100)
    analyzed_q = index_reader.analyze(q)
    for hit in hits:
      score = score_doc(analyzed_q,hit.docid,lambd)
      results.append([qid,hit.docid,score])
    # =======================

In [38]:
# Test your code
print(round(sum([item[2] for item in results]), 3))
assert round(sum([item[2] for item in results]), 3) == -160109.875

-160109.875


Write your results into a file.
Submit this file together with the completed notebook.

In [43]:
# check duplicates
check = set()
for res in results:
    if ((res[0], res[1])) in check:
        raise Exception("Error: Duplicate query-doc is found", res[0], res[1])
    check.add((res[0], res[1]))

# write results in a file
output_str = "\n".join([l[0] + "\tQ0\t" + l[1] + "\t0\t" + str(l[2]) + "\tlm_jm" for l in results])
open("lm_jm.run", "w").write(output_str)

246907

## Handing in
Submit the result file (ranked documents), the filled-in notebook, and the pdf version of your notebook:

- The result file should be named STUDENTNUMBER_FIRSTNAME_LASTNAME_lm_jm.run
- The notebook should be named STUDENTNUMBER_FIRSTNAME_LASTNAME_retrieval.ipynb
- The pdf version of your notebook should be named STUDENTNUMBER_FIRSTNAME_LASTNAME_retrieval.pdf